# Storytelling

## Objetivos:

Your data will be in front of the world's largest data science community. What questions do you want to see answered?

This dataset help to find out weather customer is Credit Risky or Credit Worthy in Banking perspective:
* Q1 - What are the factors contributing to Credit Risky customer ?
* Q2 - Behaviour of Credit Worthy Customer ?

## Características:

payment_data.csv: customer’s card payment history.
* id: customer 
* id OVDt1: number of times overdue type 1
* OVDt2: number of times overdue type 2
* OVDt3: number of times overdue type 3
* OVDsum: total overdue days paynormal: number of times normal payment
* prodcode: credit product code
* prodlimit: credit limit of product
* updatedate: account update
* date newbalance: current balance of product
*highestbalance: highest balance in history
* reportdate: date of recent payment

In [80]:
import pandas                              as pd
import numpy                               as np

import seaborn                             as sn
import matplotlib.pyplot                   as plt

from google.colab import drive

In [81]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [82]:
#Dataframe que traz as características demográficas e risco de crédito de cada consumidor
_path = '/content/gdrive/MyDrive/Colab Notebooks/Risco_de_credito/customer_data.csv'
df_customer =  pd.read_csv(_path)
display(df_customer.shape)

#Dataframe que contém o histórico de disponibilidade de crédito do cliente
_path = '/content/gdrive/MyDrive/Colab Notebooks/Risco_de_credito/payment_data.csv'
df_payment = pd.read_csv(_path)
display(df_payment.shape)

(1125, 13)

(8250, 12)

## Tratamento de dados

In [83]:
#Guarda o histórico de disponibilidade de crédito do consumidor
df_payment.groupby('id')['update_date'].count().reset_index().sort_values('update_date', ascending=False)

,id,update_date
508,58988212,55
281,54990497,48
859,58998646,45
480,58987276,39
622,58991343,39
...,...,...
977,59002046,1
541,58989236,1
332,58982507,1
763,58995389,1


In [84]:
#Os primeiros passos que iremos analisar no dataframe será vinculado a disponibilidade de crédito do produto para cada consumidor, retirando valores nulos e alocando limite zero
df_payment.head(12)

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016
5,59004828,0,0,0,0,3,10,88000.0,02/12/2016,3196.8,6193.0,15/04/2016
6,58994429,0,0,0,0,2,10,16500.0,02/12/2016,3252.0,3210.0,NaN
7,54987756,0,0,0,0,2,1,NaN,02/12/2016,365331.6,304943.0,NaN
8,58988028,0,0,0,0,4,0,NaN,02/12/2016,16795.2,28500.0,19/04/2016
9,58993180,0,0,0,0,3,6,NaN,02/12/2016,26688.0,31300.0,20/03/2016


In [85]:
df_payment['update_date'] = pd.to_datetime(df_payment['update_date'])
df_payment['report_date'] = pd.to_datetime(df_payment['report_date'])

In [86]:
#Retiramos os nulos e adicionamos zeros para limites não disponíveis
df_payment['prod_limit'] = df_payment['prod_limit'].fillna(0)

In [87]:
df_payment.isnull().sum()

id                    0
OVD_t1                0
OVD_t2                0
OVD_t3                0
OVD_sum               0
pay_normal            0
prod_code             0
prod_limit            0
update_date          26
new_balance           0
highest_balance     409
report_date        1114
dtype: int64


**highest_balance**
* Descrição: Para os valores nulos relacionados ao maior valor cadastrado no histórico do cliente, temos uma possibilidade de atuação:
  * Ordenar até a data de update da instância que possui valor nulo e cadastrar o maior valor de balance cadastrado;

In [88]:
#Para os valores nulos relacionados ao maior valor cadastrado no histórico do cliente, temos uma possibilidade de atuação:
highest_null = df_payment[df_payment['highest_balance'].isnull()][['id','update_date','highest_balance']].rename({'highest_balance':'highest_null'}, axis=1)

In [89]:
#Filtramos todos os maiores valores cadastrados não nulos e selecionamos apenas as colunas que iremos utilizar para o tratamento de dados
datas_compares = df_payment[~df_payment['highest_balance'].isnull()][['id','update_date','highest_balance']].rename({'update_date':'date_compare'}, axis=1)

In [90]:
df_tratment = datas_compares.merge(highest_null, on=['id'], how='inner')

In [91]:
df_tratment.columns

Index(['id', 'date_compare', 'highest_balance', 'update_date', 'highest_null'], dtype='object')

In [92]:
#Filtramos apenas datas de comparação menores ou iguais a data de update para atualizarmos os ultimos valores (maiores valores)
df_tratment = df_tratment[df_tratment['date_compare'] <= df_tratment['update_date']]
df_tratment.head(12)

,id,date_compare,highest_balance,update_date,highest_null
54,54991742,2005-09-06,67500.0,2006-02-11,NaN
55,54991742,2005-09-06,67500.0,2005-11-02,NaN
56,54991742,2005-11-02,12537.0,2006-02-11,NaN
57,54991742,2005-11-02,12537.0,2005-11-02,NaN
58,54991742,2005-10-01,761.0,2006-02-11,NaN
59,54991742,2005-10-01,761.0,2005-11-02,NaN
61,58993672,2015-02-28,47048.0,2015-07-22,NaN
62,58993672,2011-01-04,1500500.0,2015-07-22,NaN
63,58993672,2010-10-13,1600500.0,2015-07-22,NaN
64,58991158,2016-01-11,21400.0,2016-02-06,NaN


In [93]:
maiores_valores = df_tratment.groupby(['id','update_date'])['highest_balance'].max().reset_index()

In [94]:
maiores_valores[maiores_valores['id']==54982353]

,id,update_date,highest_balance
0,54982353,2005-09-30,700500.0
1,54982353,2006-11-21,700500.0
2,54982353,2007-12-03,775030.0


In [95]:
#Validação rapida dos dados
df_payment[(df_payment['id']==54982353) & (df_payment['highest_balance'].isnull())].sort_values('update_date')

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
8214,54982353,0,0,0,0,0,10,0.0,1994-12-18,13909.2,NaN,NaT
7749,54982353,0,0,0,0,0,10,0.0,2005-09-30,0.0,NaN,NaT
7365,54982353,0,0,0,0,1,10,0.0,2006-11-21,0.0,NaN,NaT
7234,54982353,0,0,0,0,5,10,0.0,2007-12-03,0.0,NaN,NaT


In [98]:
df_payment[(df_payment['id']==54982353) & (~df_payment['highest_balance'].isnull()) & (df_payment['update_date']>'1994-12-18')].sort_values('update_date')

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
8200,54982353,1,0,0,15,35,10,275000.0,1997-04-21,-1197.6,333140.0,2015-07-21
8143,54982353,0,0,0,0,11,2,0.0,2001-02-21,0.0,248874.0,2007-06-22
8087,54982353,0,0,0,0,35,10,5500.0,2003-05-01,0.0,4043.0,2003-11-25
8041,54982353,0,0,0,0,4,10,0.0,2003-09-17,0.0,4458.0,2006-03-28
8035,54982353,0,0,0,0,34,10,0.0,2003-10-17,0.0,5610.0,2005-05-28
7878,54982353,0,0,0,0,1,1,0.0,2005-02-16,0.0,700500.0,2008-01-18
7838,54982353,1,2,3,562,13,10,0.0,2005-04-22,0.0,5831.0,2005-07-10
7670,54982353,0,0,0,0,1,10,0.0,2006-08-01,0.0,1622.0,NaT
7233,54982353,0,0,0,0,26,10,660000.0,2007-12-03,252998.4,775030.0,2015-12-23
7235,54982353,0,0,0,0,12,10,0.0,2007-12-03,120381.6,546657.0,2011-05-21
